## Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
import io
import cv2
import json

## Data preprocessing
Block of code about transforming the data from .mkv Azure Kinect files to depth and rgb pictures and video.

In [2]:
img = cv2.imread('data/depth0177.png', -1)

In [3]:
with open('data/mkv_meta.json') as f:
    intrinsic_params = json.load(f)

In [4]:
camera_intrinsic_params = {}
tmp = intrinsic_params['depth_intrinsics'].copy()
tmp.update(intrinsic_params['depth_resolution'].copy())
camera_intrinsic_params['height'] = tmp['h']
camera_intrinsic_params['width'] = tmp['w']
camera_intrinsic_params['fx'] = tmp['fx']
camera_intrinsic_params['fy'] = tmp['fy']
camera_intrinsic_params['cx'] = tmp['cx']
camera_intrinsic_params['cy'] = tmp['cy']

In [5]:
camera_intrinsic_params

{'height': 576,
 'width': 640,
 'fx': 505.05084228515625,
 'fy': 504.9956359863281,
 'cx': 338.0733642578125,
 'cy': 338.06817626953125}

In [6]:
o3d_depth = o3d.geometry.Image(img)
# camera_intrinsic = o3d.camera.PinholeCameraIntrinsic(
#     o3d.camera.PinholeCameraIntrinsicParameters.Kinect2ColorCameraDefault)
# camera_intrinsic = o3d.io.read_pinhole_camera_intrinsic('data/mkv_meta.json')
camera_intrinsic = o3d.camera.PinholeCameraIntrinsic(**camera_intrinsic_params)

In [7]:
np.unique(np.asarray(o3d_depth))

array([   0,  653,  701, ..., 4355, 4361, 5855], dtype=uint16)

In [8]:
pcd = o3d.geometry.PointCloud.create_from_depth_image(o3d_depth, camera_intrinsic)

In [9]:
pcd

PointCloud with 272925 points.

In [10]:
# Estimate normals
pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

In [11]:
# o3d.visualization.draw_geometries([pcd])

In [16]:
trm, _ = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd)

In [17]:
np.asarray(trm.triangles)

array([[  558,   388,   432],
       [  558,   530,   388],
       [  558,   481,   530],
       ...,
       [37423, 37583, 37529],
       [37423, 37417, 37583],
       [37532, 37390, 37424]], dtype=int32)

In [18]:
trm.compute_vertex_normals()

TriangleMesh with 37685 points and 75200 triangles.

In [21]:
trm.compute_triangle_normals()

TriangleMesh with 37685 points and 75200 triangles.

In [22]:
o3d.visualization.draw_geometries([trm])

In [22]:
mesh_box = o3d.geometry.TriangleMesh.create_box(width=1.0,
                                                height=1.0,
                                                depth=1.0)

In [13]:
print(np.asarray(pcd.points).shape)

(272925, 3)


In [11]:
o3d.io.write_point_cloud('data/depth0177.ply', pcd, print_progress=True)

True

In [15]:
pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

PointCloud with 272925 points.

In [ ]:
o3d.visualization.draw_geometries([pcd])

In [3]:
pcd = o3d.io.read_point_cloud('data/depth0177.ply')